In [6]:
import os, json
from tqdm import tqdm
from src.intension import Intension

In [9]:
MODELS = [ 
    # Intension(model_name="google/gemma-7b-it"),
    # Intension(model_name="gpt-3.5-turbo"), 
    # Intension(model_name="gpt-4-0125-preview"), 
    # Intension(model_name="mistralai/Mistral-7B-Instruct-v0.2"),
    # Intension(model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"),
    # Intension(model_name="meta-llama/Llama-2-70b-chat-hf"), 
    Intension(model_name="claude-3-opus-20240229"),
]

DATA = json.load(open('experiments/caligraph_statements.json', 'r'))

BATCH_SIZE = 1

In [10]:
for model in MODELS:
    model_name = model.model_name.split("/")[-1]
    filename = f'experiments/{model_name}-caligraph.json'
    results = []
    queries = [
        {
            "predicate": datum["predicate"]["label"],
            "arguments": ", ".join([ arg["label"] for arg in datum["arguments"] ]),
            "world": " ".join([ arg["description"] for arg in datum["arguments"] ]),
            "actual": datum["in_extension"]
        }
        for datum in DATA
    ]
    batches = [ queries[i:i+BATCH_SIZE] for i in range(0, len(queries), BATCH_SIZE) ] 
    for batch in tqdm(batches, desc=f'{model_name:30}', total=len(batches)):
        response = model.chain.batch(batch)
        for i, result in enumerate(response):
            result["rationale"] = result["text"]["rationale"]
            result["predicted"] = result["text"]["answer"]
            result.pop("text")
        results.extend(response)
    json.dump(results, open(filename, "w+"))

claude-3-opus-20240229        :   0%|          | 0/764 [00:00<?, ?it/s]

claude-3-opus-20240229        : 100%|██████████| 764/764 [3:50:01<00:00, 18.07s/it]  
